以下代码提供了Criteo数据集在垂直场景下的自定义DataBuilder，构造databuilder_dict以供除多元模型调用。为简化问题，该databuilder_dict仅采取四种特征与标签列。
由于Criteo数据集与Yahoo数据集具有同质性且数据集过大，代码运行时间过长，所以并未在criteo数据集上进行实验测试。

In [ ]:
# Copyright 2022 Ant Group Co., Ltd.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Author: Yuanran Song
# E-mail: 809127446@qq.com

In [ ]:
# load_ext autoreload
# autoreload 2
import numpy as np
import secretflow as sf
import tempfile
import tensorflow as tf
import math
import pandas as pd


print('The version of SecretFlow:{}', format(sf.__version__))

sf.shutdown()
sf.init(['alice', 'bob', 'charlie'], address="local", log_to_driver=False)
alice, bob, charlie = sf.PYU('alice'), sf.PYU('bob'), sf.PYU('charlie')


user_data = pd.read_csv('/media/whb/Elements/secretflow/train(100).txt', delimiter="\t", header=None)
user_data.to_csv('/media/whb/Elements/secretflow/train(100).csv', index=False, encoding="utf-8")
user_data = user_data.drop(columns=user_data.columns[[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]])
user_data.columns = ['label', 'I1', 'I2', 'C1', 'C2']##仅采取四种特征与标签列
# print(user_data)


# alice: C1, C2
alice_df = user_data.drop(columns=user_data.columns[[0, 1, 2]])
# bob: label, I1, I2
bob_df = user_data.drop(columns=user_data.columns[[3, 4]])

# print(alice_df)
# print(bob_df)

alice_df['C1'] = alice_df['C1'].astype("string")
alice_df['C2'] = alice_df['C2'].astype("string")
alice_dict = dict(alice_df)
data_set = tf.data.Dataset.from_tensor_slices(alice_dict).batch(32).repeat(1)
# print(data_set)


# print(bob)
label = bob_df['label']
data = bob_df.drop(columns='label')
##可以作为输入传入SLModel进行建模

def _parse_bob(row_sample, label):
    import tensorflow as tf

    y_t = label
    y = tf.expand_dims(
        tf.where(
            y_t > 3,
            tf.ones_like(y_t, dtype=tf.float32),
            tf.zeros_like(y_t, dtype=tf.float32),
        ),
        axis=1
    )
    return row_sample, y


bob_dict = tuple([dict(data), label])
data_set = tf.data.Dataset.from_tensor_slices(bob_dict).batch(32).repeat(1)
data_set = data_set.map(_parse_bob)
next(iter(data_set))

# print(next(iter(data_set)))


# warp DataBuilder

# alice
def create_dataset_builder_alice(
        batch_size=128,
        repeat_count=5,
):
    def dataset_builder(x):
        import pandas as pd
        import tensorflow as tf

        x = [dict(t) if isinstance(t, pd.DataFrame) else t for t in x]
        x = x[0] if len(x) == 1 else tuple(x)
        data_set = (
            tf.data.Dataset.from_tensor_slices(x).batch(batch_size).repeat(repeat_count)
        )
        return data_set

    return dataset_builder


# bob
def create_dataset_builder_bob(
        batch_size=128,
        repeat_count=5,
):
    def _parse_bob(row_sample, label):
        import tensorflow as tf

        y_t = label["type"]
        y = tf.expand_dims(
            tf.where(
                y_t > 3,
                tf.ones_like(y_t, dtype=tf.float32),
                tf.zeros_like(y_t, dtype=tf.float32),
            ),
            axis=1,
        )
        return row_sample, y

    def dataset_builder(x):
        import pandas as pd
        import tensorflow as tf

        x = [dict(t) if isinstance(t, pd.DataFrame) else t for t in x]
        x = x[0] if len(x) == 1 else tuple(x)
        data_set = (
            tf.data.Dataset.from_tensor_slices(x).batch(batch_size).repeat(repeat_count)
        )

        data_set = data_set.map(_parse_bob)

        return data_set

    return dataset_builder


# databuilder_dict
data_builder_dict = {
    alice: create_dataset_builder_alice(
        batch_size=128,
        repeat_count=5,
    ),
    bob: create_dataset_builder_bob(
        batch_size=128,
        repeat_count=5,
    ),
}
